In [68]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os, time
from IPython.display import display, Markdown

In [49]:
options = webdriver.ChromeOptions()
options.add_argument(' — ignore-certificate-errors')
options.add_argument(' — incognito')
options.add_argument(' — headless')

In [50]:
driver = webdriver.Chrome('./driver/chromedriver', options=options)
dir_path = os.getcwd()
url = f'file://{dir_path}/index.html'
driver.get(url)

/var/folders/_t/gs7cg9h922s7rt1lrsw8d50w0000gn/T/ipykernel_10954/1858395511.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver', options=options)


In [51]:
countries = ['australia','india','us']
number_of_colors = [2, 3, 4]
algorithms = ['backtracking','forward_checking']
heuristics = ['mrv','dh','noh']


In [52]:
mapping_ids = {
    'number_of_colors': 'numColors',
    'countries': 'map-option',
    'algorithms': 'search-option',
    'heuristics': 'forward_checking-options-container'
}   
numeber_of_tries = 5

In [53]:
def choose_color(num_of_colors):
    input_element = driver.find_element(By.ID,mapping_ids['number_of_colors'])    
    input_element.clear()
    input_element.send_keys(num_of_colors)

def choose_country(value):
    dropdown_element = driver.find_element(By.ID,mapping_ids['countries'])    
    dropdown_element = Select(dropdown_element)
    dropdown_element.select_by_value(value)

def choose_algorithm(value):
    dropdown_element  = driver.find_element(By.ID,mapping_ids['algorithms'])    
    dropdown_element = Select(dropdown_element)
    dropdown_element.select_by_value(value)

def choose_heuristic(value):
    div_container  = driver.find_element(By.ID,mapping_ids['heuristics'])    
    radio_button = div_container.find_element(By.ID,value)
    radio_button.click()

In [54]:
def run_csp():
    button_element = driver.find_element(By.ID,'runCSP')
    button_element.click()

def csp_executing():
    status = driver.find_element(By.ID,'status')
    if status.text.strip() =='':
        return True
    return False

def get_execution_time():
    execution_time = driver.find_element(By.ID,'timer')
    return float(execution_time.text.replace('Execution time: ','').replace(' ms',''))

def csp_solved_status():
    status = driver.find_element(By.ID,'status')
    if status.text == 'Solution Status: Found':
        return 'found'
    return 'not_found'

In [55]:
def run_backtracking():
    results = []
    for i in range(numeber_of_tries):
        for country in countries:
            for color in number_of_colors: 
                result = {}
                result['num_of_colors'] = color
                result['country'] = country
                result['algorithm'] = 'backtracking'
                choose_color(color)
                choose_country(country)
                choose_algorithm('backtracking')
                run_csp()
                while csp_executing():
                    time.sleep(2)
                result['execution_time'] = get_execution_time()
                result['execution_id'] = i+1
                result['solved_status'] = csp_solved_status()
                results.append(result)
                driver.refresh()
                time.sleep(1)
    return results

In [56]:
def run_forward_checking():
    results = []
    for i in range(numeber_of_tries):
        for country in countries:
            for color in number_of_colors:
                for heuristic in heuristics: 
                    result = {}
                    result['num_of_colors'] = color
                    result['country'] = country
                    choose_color(color)
                    choose_country(country)
                    choose_algorithm('forward_checking')
                    choose_heuristic(heuristic)
                    if heuristic =='noh':
                        result['algorithm'] = 'forward_checking_without_heuristic'
                    else:
                        result['algorithm'] = f'forward_checking_{heuristic}'
                    run_csp()
                    while csp_executing():
                        time.sleep(2)
                    result['execution_time'] = get_execution_time()
                    result['execution_id'] = i+1
                    result['solved_status'] = csp_solved_status()
                    results.append(result)
                    driver.refresh()
                    time.sleep(1)
    return results

In [57]:
backtracking_results = run_backtracking()

In [58]:
forward_checking_results = run_forward_checking()

In [59]:
df = pd.DataFrame([item for sublist in [backtracking_results,forward_checking_results] for item in sublist])

In [79]:
def show_agg(df,country):
    df = df[df['country']==country]
    display(Markdown(f'## Country: {country.upper()}, Map Coloring Average Execution Time and Number of Executions'))
    grouped_df = df.groupby(['country','num_of_colors','algorithm','solved_status'])
    result = grouped_df.agg({'execution_time': 'mean', 'execution_id': 'count'}).rename(columns={'execution_time': 'avg execution time (ms)', 'execution_id': 'number of executions'})
    return result

In [75]:
show_agg(df,'australia')

## Country: Australia, Map Coloring Average Execution Time and Number of Executions

avg execution time (ms)  \
country   num_of_colors algorithm                          solved_status                            
australia 2             backtracking                       not_found                       128.92   
                        forward_checking_dh                not_found                        87.56   
                        forward_checking_mrv               not_found                        67.94   
                        forward_checking_without_heuristic not_found                        80.78   
          3             backtracking                       found                           113.38   
                        forward_checking_dh                found                            79.52   
                        forward_checking_mrv               found                           102.30   
                        forward_checking_without_heuristic found                            87.28   
          4             backtracking                       found                           120.74   
                        forward_checking_dh                found                            85.48   
                        forward_checking_mrv               found                            73.06   
                        forward_checking_without_heuristic found                            69.30   

                                                                          number of executions  
country   num_of_colors algorithm                          solved_status                        
australia 2             backtracking                       not_found                         5  
                        forward_checking_dh                not_found                         5  
                        forward_checking_mrv               not_found                         5  
                        forward_checking_without_heuristic not_found                         5  
          3             backtracking                       found                             5  
                        forward_checking_dh                found                             5  
                        forward_checking_mrv               found                             5  
                        forward_checking_without_heuristic found                             5  
          4             backtracking                       found                             5  
                        forward_checking_dh                found                             5  
                        forward_checking_mrv               found                             5  
                        forward_checking_without_heuristic found                             5

In [76]:
show_agg(df,'india')

## Country: India, Map Coloring Average Execution Time and Number of Executions

avg execution time (ms)  \
country num_of_colors algorithm                          solved_status                            
india   2             backtracking                       not_found                      1336.86   
                      forward_checking_dh                not_found                      1373.48   
                      forward_checking_mrv               not_found                      1239.32   
                      forward_checking_without_heuristic not_found                      1318.86   
        3             backtracking                       not_found                      1357.40   
                      forward_checking_dh                not_found                      1313.92   
                      forward_checking_mrv               not_found                      1247.30   
                      forward_checking_without_heuristic not_found                      1395.88   
        4             backtracking                       found                          1289.94   
                      forward_checking_dh                found                          1341.80   
                      forward_checking_mrv               found                          1390.54   
                      forward_checking_without_heuristic found                          1400.24   

                                                                        number of executions  
country num_of_colors algorithm                          solved_status                        
india   2             backtracking                       not_found                         5  
                      forward_checking_dh                not_found                         5  
                      forward_checking_mrv               not_found                         5  
                      forward_checking_without_heuristic not_found                         5  
        3             backtracking                       not_found                         5  
                      forward_checking_dh                not_found                         5  
                      forward_checking_mrv               not_found                         5  
                      forward_checking_without_heuristic not_found                         5  
        4             backtracking                       found                             5  
                      forward_checking_dh                found                             5  
                      forward_checking_mrv               found                             5  
                      forward_checking_without_heuristic found                             5

In [80]:
show_agg(df,'us')

## Country: US, Map Coloring Average Execution Time and Number of Executions

avg execution time (ms)  \
country num_of_colors algorithm                          solved_status                            
us      2             backtracking                       not_found                       144.84   
                      forward_checking_dh                not_found                       128.04   
                      forward_checking_mrv               not_found                       128.00   
                      forward_checking_without_heuristic not_found                        87.36   
        3             backtracking                       not_found                      7010.12   
                      forward_checking_dh                not_found                       102.38   
                      forward_checking_mrv               not_found                       113.48   
                      forward_checking_without_heuristic not_found                     20711.12   
        4             backtracking                       found                           776.80   
                      forward_checking_dh                found                            94.96   
                      forward_checking_mrv               found                           103.32   
                      forward_checking_without_heuristic found                           578.10   

                                                                        number of executions  
country num_of_colors algorithm                          solved_status                        
us      2             backtracking                       not_found                         5  
                      forward_checking_dh                not_found                         5  
                      forward_checking_mrv               not_found                         5  
                      forward_checking_without_heuristic not_found                         5  
        3             backtracking                       not_found                         5  
                      forward_checking_dh                not_found                         5  
                      forward_checking_mrv               not_found                         5  
                      forward_checking_without_heuristic not_found                         5  
        4             backtracking                       found                             5  
                      forward_checking_dh                found                             5  
                      forward_checking_mrv               found                             5  
                      forward_checking_without_heuristic found                             5

In [81]:
df

,num_of_colors,country,algorithm,execution_time,execution_id,solved_status
0,2,australia,backtracking,208.6,1,not_found
1,3,australia,backtracking,92.1,1,found
2,4,australia,backtracking,115.9,1,found
3,2,india,backtracking,1833.2,1,not_found
4,3,india,backtracking,916.9,1,not_found
...,...,...,...,...,...,...
175,3,us,forward_checking_dh,104.2,5,not_found
176,3,us,forward_checking_without_heuristic,20647.2,5,not_found
177,4,us,forward_checking_mrv,95.8,5,found
178,4,us,forward_checking_dh,95.1,5,found
